### Risk management module for swing trading

This module is to calculate value at risk of the portfolio  and expected short fall for the last 150 days. And Comparison portfolio VaR and Expected Short fall with that of S&P500 

Comparision of portfolio returns with s&p500 returns

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for yahoofinancials: filename=yahoofinancials-1.14-py3-none-any.whl size=28627 sha256=5a43fb414503413c792648b21aa18ec4f0d61e31320213911e236ee443ddda1a
  Stored in directory: /root/.cache/pip/wheels/32/a3/b6/b5b33187f2d42f095fecc236b957b46173fa09e78a106e309f
Successfully built yahoofinancials


In [ ]:
# Third-party libraries
import numpy as np 
import pandas as pd
import matplotlib. pyplot as plt 
from scipy.stats import norm
import scipy
import yfinance as yf 
from yahoofinancials import YahooFinancials
from tabulate import tabulate 
import plotly.graph_objects as go

# Standard library
from typing import List

In [ ]:
# Portfolio assets
stocks = ["AAPL", "MSFT", "GOOG", "AMZN"]

# Weights for equally weighted portfolio
weights = np.array([1/4]*4)

In [ ]:
def get_portfolio_asset_returns(stocks:List[str], start:str, end:str) -> pd.DataFrame:
  """Gets portfolio asset daily returns for a give period"""
  stock_data = yf.download(stocks, start, end, progress=False)
  stock_data = stock_data['Close']
  returns = stock_data.pct_change()
  return returns

In [ ]:
# Time horizon
start = '2018-01-01'
end = '2023-04-01'

# Stock returns
returns = get_portfolio_asset_returns(stocks=stocks, start=start, end=end)
returns.tail()

,AAPL,AMZN,GOOG,MSFT
Date,,,,
2023-03-27,-0.012293,-0.000917,-0.028286,-0.014934
2023-03-28,-0.003980,-0.008160,-0.016495,-0.004161
2023-03-29,0.019791,0.030954,0.005328,0.019184
2023-03-30,0.009890,0.017456,-0.005692,0.012620
2023-03-31,0.015644,0.012647,0.026451,0.014962


In [ ]:
# S&P500 returns
snp500_returns = get_portfolio_asset_returns(stocks=["^GSPC"], start=start, end=end)
snp500_returns.tail()

Date
2023-03-27    0.001647
2023-03-28   -0.001574
2023-03-29    0.014237
2023-03-30    0.005715
2023-03-31    0.014437
Name: Close, dtype: float64

In [ ]:
# Plotting stock daily returns against S&P500 returns
fig=go.Figure()
for ticker in stocks:
  fig.add_trace(go.Scatter(x=returns.index, y=returns[ticker], name=ticker))
fig.add_trace(go.Scatter(x=snp500_returns.index, y=snp500_returns.values, name="S&P 500"))
fig.update_layout(xaxis_title='Dates',yaxis_title='Daily returns',
        title=dict(text='Stocks daily returns', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

In [ ]:
def var_historical(returns, confidence_level=0.05):
  """Calculates historical VAR with a given conficence level, non-parametric metric"""
	
  return returns.quantile(confidence_level, interpolation='linear')
     

In [ ]:
def es_historical(returns, confidence_level=0.05):
  """Calculates Expected Shortfall (ES)"""
	
  # Calculating VaR
  var = var_historical(returns, confidence_level)
	
	# Given the VAR, ES (AVAR, CVAR) is the average of the worst losses
  return returns[returns.lt(var)].mean()

In [ ]:
snp500_returns.describe()

count    1320.000000
mean        0.000413
std         0.013640
min        -0.119841
25%        -0.005327
50%         0.000874
75%         0.007181
max         0.093828
Name: Close, dtype: float64

In [ ]:
# Calculating 95% VAR and 95% ES for the last 150 days
snp500_var_95 = []
snp500_var_99 = []
snp500_es_95 = []
snp500_es_99 = []

for i in range(len(snp500_returns) -150, len(snp500_returns)):
  # Calculating VAR
  snp500_var_95.append(-var_historical(snp500_returns[0:i].dropna(), confidence_level=0.05))
  snp500_var_99.append(-var_historical(snp500_returns[0:i].dropna(), confidence_level=0.01))
  
  # Calcularing ES
  snp500_es_95.append(-es_historical(snp500_returns[0:i].dropna(), confidence_level=0.05))
  snp500_es_99.append(-es_historical(snp500_returns[0:i].dropna(), confidence_level=0.01))

In [ ]:
# Plotting VAR and AVAR for S&P500 index
fig=go.Figure()
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_var_95, name="95% VAR"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_var_99, name="99% VAR"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_es_95, name="95% ES"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_es_99, name="99% ES"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_returns[len(snp500_returns) -150: len(snp500_returns)], name="S&P500 daily returns"))
fig.update_layout(xaxis_title='Last 150 days',yaxis_title='Expected loss for the given risk measure',
        title=dict(text='S&P500 VAR and Expected Shortfall', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

In [ ]:
# Portfolio asset statistical properties
returns.describe()

,AAPL,AMZN,GOOG,MSFT
count,1320.000000,1320.000000,1320.000000,1320.000000
mean,0.001235,0.000675,0.000707,0.001108
std,0.020868,0.022649,0.020009,0.019544
min,-0.128647,-0.140494,-0.111008,-0.147390
25%,-0.009103,-0.010744,-0.008791,-0.008464
50%,0.001244,0.001229,0.001072,0.001157
75%,0.012455,0.012088,0.011043,0.011076
max,0.119808,0.135359,0.104485,0.142169


In [ ]:
# Portfolio returns
returns_p = returns * weights
returns_p = returns_p.dropna().sum(axis=1)
returns_p

Date
2018-01-03    0.008417
2018-01-04    0.005386
2018-01-05    0.013629
2018-01-08    0.004001
2018-01-09    0.000817
                ...   
2023-03-27   -0.014108
2023-03-28   -0.008199
2023-03-29    0.018814
2023-03-30    0.008569
2023-03-31    0.017426
Length: 1320, dtype: float64

In [ ]:
# Plotting portfolio daily returns against S&P500 returns
fig=go.Figure()
fig.add_trace(go.Scatter(x=snp500_returns.index, y=snp500_returns.values, name="S&P 500"))
fig.add_trace(go.Scatter(x=returns_p.index, y=returns_p.values, name="Portfolio"))
fig.update_layout(xaxis_title='Dates',yaxis_title='Daily returns',
        title=dict(text='Portfolio vs S&P 500 daily returns', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

In [ ]:
# Calculating 95% VAR and 95% ES for the last 150 days
portfolio_var_95 = []
portfolio_var_99 = []
portfolio_es_95 = []
portfolio_es_99 = []

for i in range(len(returns_p) -150, len(returns_p)):
  # Calculating VAR
  portfolio_var_95.append(-var_historical(returns_p[0:i].dropna(), confidence_level=0.05))
  portfolio_var_99.append(-var_historical(returns_p[0:i].dropna(), confidence_level=0.01))
  
  # Calcularing Expected Shortfall
  portfolio_es_95.append(-es_historical(returns_p[0:i].dropna(), confidence_level=0.05))
  portfolio_es_99.append(-es_historical(returns_p[0:i].dropna(), confidence_level=0.01))

In [ ]:
# Plotting VAR and AVAR for our portfolio
fig=go.Figure()
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_var_95, name="95% VAR"))
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_var_99, name="99% VAR"))
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_es_95, name="95% ES"))
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_es_99, name="99% ES"))
fig.update_layout(xaxis_title='Last 150 days',yaxis_title='Expected loss for the given risk measure',
        title=dict(text='Portfolio VAR and Expected Shortfall', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

In [ ]:
# VAR for our portfolio and S&P 500
fig=go.Figure()
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_var_95, name="95% VAR: portfolio"))
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_var_99, name="99% VAR: portfolio"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_var_95, name="95% VAR: S&P500"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_var_99, name="99% VAR: S&P500"))
fig.update_layout(xaxis_title='Last 150 days',yaxis_title='Expected loss for the given risk measure',
        title=dict(text='95%/99% VAR: S&P500 vs. our portfolio', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

In [ ]:
# ES for our portfolio and S&P 500
fig=go.Figure()
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_es_95, name="95% ES: portfolio"))
fig.add_trace(go.Scatter(x=list(range(150)), y=portfolio_es_99, name="99% ES: portfolio"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_es_95, name="95% ES: S&P500"))
fig.add_trace(go.Scatter(x=list(range(150)), y=snp500_es_99, name="99% ES: S&P500"))
fig.update_layout(xaxis_title='Last 150 days',yaxis_title='Expected loss for the given risk measure',
        title=dict(text='95%/99% ES: S&P500 vs. our portfolio', x=0.5, y=0.87, font=dict(size=22,color='Navy')))
fig.show()

In [ ]:
# Expected daily return for portfolio 1 
portfolio_mean_1 = returns_p.mean()
print(f"Expected daily return for portfolio 1: {np.round(portfolio_mean_1*100, 4)} %")

Expected daily return for portfolio 1: 0.0932 %


In [ ]:
returns.mean()

AAPL    0.001235
AMZN    0.000675
GOOG    0.000707
MSFT    0.001108
dtype: float64

In [ ]:

# Covariance matrix for assets in portfolio 
cov_matrix_1 = returns.cov()
cov_matrix_1

,AAPL,AMZN,GOOG,MSFT
AAPL,0.000435,0.000305,0.000289,0.000312
AMZN,0.000305,0.000513,0.000311,0.000316
GOOG,0.000289,0.000311,0.000400,0.000312
MSFT,0.000312,0.000316,0.000312,0.000382
